In [2]:
!pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
#/content/drive/MyDrive/grammer checker/Book1.xlsx

In [4]:
import pandas as pd

# Replace 'your_file.xlsx' with the actual filename
file_name = '/content/drive/MyDrive/grammer checker/Book1.xlsx'

# Load the Excel file
df = pd.read_excel(file_name)

# Display the first few rows of the dataset
print(df.head())


                       Sentence  Label  Error_Type Subject          Verb  \
0   அவன் பள்ளிக்குச் செல்கிறான்      1           0    அவன்    செல்கிறான்   
1     அவன் பள்ளியில் செல்கிறான்      0           2    அவன்    செல்கிறான்   
2   அவள் படிக்கிறாள் புத்தகத்தை      0           3    அவள்   படிக்கிறாள்   
3  அவள் காய்கறிகளை வெட்டுகிறாள்      1           0    அவள்  வெட்டுகிறாள்   
4     அவன் தண்ணீரை குடிக்கிறான்      1           0    அவன்  குடிக்கிறான்   

       Object  2  Sentence_Length            Corrected Sentence  
0   பள்ளிக்கு  -                3   அவன் பள்ளிக்குச் செல்கிறான்  
1   பள்ளியில்  -                3   அவன் பள்ளிக்குச் செல்கிறான்  
2  புத்தகத்தை  -                3   அவள் புத்தகத்தை படிக்கிறாள்  
3  காய்கறிகளை  -                3  அவள் காய்கறிகளை வெட்டுகிறாள்  
4     தண்ணீரை  -                3     அவன் தண்ணீரை குடிக்கிறான்  


In [5]:
import pandas as pd
from google.colab import files
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset

In [6]:
# Step 4: Data Preprocessing
# Map labels to integers
error_types = df['Error_Type'].unique()
label_mapping = {label: idx for idx, label in enumerate(error_types)}
df['Label'] = df['Error_Type'].map(label_mapping)

# Tokenization using BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Function to tokenize text
def tokenize_data(texts):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [7]:
# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Sentence'],
    df['Label'],
    test_size=0.2,
    random_state=42
)

In [8]:
# Tokenize training and validation texts
train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)

In [9]:
# Step 5: Create a Dataset Class
class TamilGrammarDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = TamilGrammarDataset(train_encodings, train_labels.tolist())
val_dataset = TamilGrammarDataset(val_encodings, val_labels.tolist())


In [10]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=len(error_types)
)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:


# Disable Weights & Biases
import os
os.environ["WANDB_DISABLED"] = "true"

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
# Step 8: Train the Model
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.743833
2,No log,0.550193
3,No log,0.509344


TrainOutput(global_step=75, training_loss=0.6126862589518229, metrics={'train_runtime': 226.0058, 'train_samples_per_second': 2.628, 'train_steps_per_second': 0.332, 'total_flos': 7631385418800.0, 'train_loss': 0.6126862589518229, 'epoch': 3.0})

In [13]:
# Step 10: Evaluate the Model
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


Evaluation results: {'eval_loss': 0.5093436241149902, 'eval_runtime': 2.8599, 'eval_samples_per_second': 17.483, 'eval_steps_per_second': 2.448, 'epoch': 3.0}


In [16]:
# Step 11: Define Prediction Function
def predict_grammar(sentence):
    # Ensure model and inputs are on the same device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize input
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to device

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_label = torch.argmax(outputs.logits, dim=1).item()

    # Map label to error type and corrected sentence
    predicted_error = [key for key, value in label_mapping.items() if value == predicted_label]
    predicted_error_type = predicted_error[0] if predicted_error else "Unknown"

    # Find the corrected sentence if available
    corrected_sentence = df[df['Label'] == predicted_label]['Corrected Sentence'].iloc[0]

    return predicted_error_type, corrected_sentence



In [19]:
# Step 12: Test the Model
sample_sentence = "அவன் தண்ணீரை குடிக்கிறாள்"
predicted_error = predict_grammar(sample_sentence)
print(f"Predicted error type for '{sample_sentence}': {predicted_error}")

Predicted error type for 'அவன் தண்ணீரை குடிக்கிறாள்': (1, 'அவன் தண்ணீரை குடிக்கிறான்')
